## Preparação

Vamos carregar o que criamos no Bloco 1:

In [7]:
import json
import requests
from pathlib import Path
from time import sleep

# Carregar configuração
with open("config.json") as f:
    config = json.load(f)

print("Configuração carregada!")
print(f"Base URL: {config['tmdb']['base_url']}")

Configuração carregada!
Base URL: https://api.themoviedb.org/3


## Passo 1: Buscar Lista de Filmes Populares

Em vez de buscar um filme por vez, vamos descobrir quais filmes são populares.

### Entender o endpoint /movie/popular

A API do TMDb tem diferentes **endpoints** (caminhos):

- `/movie/{id}` - Busca um filme específico (usamos no Bloco 1)
- `/movie/popular` - Lista filmes populares
- `/movie/top_rated` - Lista filmes bem avaliados
- `/movie/now_playing` - Lista filmes em cartaz

Vamos usar o `/movie/popular` para descobrir filmes interessantes.

In [5]:
# Construir URL
url = f"{config['tmdb']['base_url']}/movie/popular"

print("Endpoint de filmes populares:")
print(url)

Endpoint de filmes populares:
https://api.themoviedb.org/3/movie/popular


In [ ]:
# Fazer requisição
response = requests.get(
    url,
    params={
        'api_key': config['tmdb']['api_key'],
        'language': config['tmdb']['language'],
        'page': 1  # Primeira página
    }
)

print(f"Status: {response.status_code}")

if response.status_code == 200:
    populares = response.json()
    print("Dados recebidos com sucesso!")

Status: 401


### Explorar estrutura da resposta

A resposta tem uma estrutura diferente do que vimos antes:

In [4]:
print("Estrutura da resposta:")
print()
for chave in populares.keys():
    valor = populares[chave]
    tipo = type(valor).__name__
    
    if tipo == 'list':
        print(f"{chave}: lista com {len(valor)} itens")
    else:
        print(f"{chave}: {valor}")

Estrutura da resposta:



NameError: name 'populares' is not defined

### Entender paginação

A API retorna resultados em **páginas**:
- Cada página tem até 20 filmes
- `page`: página atual
- `total_pages`: quantas páginas existem
- `total_results`: quantos filmes no total

In [5]:
print(f"Página atual: {populares['page']}")
print(f"Total de páginas: {populares['total_pages']}")
print(f"Total de resultados: {populares['total_results']}")
print(f"Filmes nesta página: {len(populares['results'])}")

NameError: name 'populares' is not defined

### Ver os filmes

Vamos ver os primeiros 5 filmes:

In [6]:
print("Top 5 filmes populares:")
print()

for i, filme in enumerate(populares['results'][:5], 1):
    print(f"{i}. {filme['title']} ({filme['release_date'][:4]})")
    print(f"   ID: {filme['id']}")
    print(f"   Nota: {filme['vote_average']}/10")
    print()

Top 5 filmes populares:



NameError: name 'populares' is not defined

## Passo 2: Função para Buscar Filmes

Vamos criar uma função para não repetir código.

### Por que usar funções?

Funções evitam repetição:
- Escrevemos uma vez
- Usamos várias vezes
- Fica mais fácil de manter

In [7]:
def buscar_filme_por_id(movie_id: int) -> dict:
    """
    Busca um filme específico pelo ID.
    
    Args:
        movie_id: ID do filme no TMDb
    
    Returns:
        Dicionário com dados do filme
    """
    url = f"{config['tmdb']['base_url']}/movie/{movie_id}"
    
    response = requests.get(
        url,
        params={
            'api_key': config['tmdb']['api_key'],
            'language': config['tmdb']['language']
        }
    )
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro ao buscar filme {movie_id}: {response.status_code}")
        return None

print("Função buscar_filme_por_id criada!")

Função buscar_filme_por_id criada!


### Testar a função:

In [8]:
# Testar com Matrix (ID: 603)
matrix = buscar_filme_por_id(603)

if matrix:
    print(f"Título: {matrix['title']}")
    print(f"Ano: {matrix['release_date'][:4]}")
    print(f"Nota: {matrix['vote_average']}/10")

Erro ao buscar filme 603: 401


## Passo 3: Buscar Vários Filmes

Agora vamos buscar detalhes de vários filmes da lista de populares.

### Extrair IDs da lista

Primeiro, vamos pegar os IDs dos filmes populares:

In [9]:
# Pegar IDs dos 10 primeiros filmes
movie_ids = [filme['id'] for filme in populares['results'][:10]]

print(f"Temos {len(movie_ids)} IDs de filmes:")
print(movie_ids)

NameError: name 'populares' is not defined

### Buscar todos os filmes

Vamos buscar detalhes de cada um:

In [10]:
filmes_detalhados = []

print("Buscando detalhes dos filmes...")
print()

for i, movie_id in enumerate(movie_ids, 1):
    print(f"{i}/{len(movie_ids)}: Buscando filme {movie_id}...", end=" ")
    
    filme = buscar_filme_por_id(movie_id)
    
    if filme:
        filmes_detalhados.append(filme)
        print(f"OK - {filme['title']}")
    else:
        print("ERRO")
    
    # Aguardar um pouco para não sobrecarregar a API
    sleep(0.5)

print()
print(f"Total coletado: {len(filmes_detalhados)} filmes")

Buscando detalhes dos filmes...



NameError: name 'movie_ids' is not defined

### Por que usar sleep()?

APIs têm **rate limits** (limites de taxa):
- Você não pode fazer milhões de requisições por segundo
- `sleep(0.5)` aguarda meio segundo
- Isso evita ser bloqueado pela API
- É uma questão de respeito e boas práticas

## Passo 4: Salvar Dados Coletados

Vamos organizar e salvar os filmes que coletamos.

### Salvar cada filme individualmente:

In [11]:
# Verificar se pasta dados existe
dados_path = Path("dados")
dados_path.mkdir(exist_ok=True)

print(f"Salvando {len(filmes_detalhados)} filmes...")

for filme in filmes_detalhados:
    movie_id = filme['id']
    arquivo = dados_path / f"filme_{movie_id}.json"
    
    with open(arquivo, "w", encoding="utf-8") as f:
        json.dump(filme, f, indent=2, ensure_ascii=False)
    
    print(f"  Salvo: {arquivo.name}")

print()
print("Todos os filmes salvos!")

Salvando 0 filmes...

Todos os filmes salvos!


### Criar um índice

Vamos criar um arquivo com a lista de todos os filmes:

In [12]:
# Criar resumo de cada filme
indice = []

for filme in filmes_detalhados:
    resumo = {
        'id': filme['id'],
        'title': filme['title'],
        'release_date': filme['release_date'],
        'vote_average': filme['vote_average'],
        'vote_count': filme['vote_count']
    }
    indice.append(resumo)

print(f"Índice criado com {len(indice)} filmes")

Índice criado com 0 filmes


In [13]:
# Salvar índice
indice_path = dados_path / "indice_filmes.json"

with open(indice_path, "w", encoding="utf-8") as f:
    json.dump(indice, f, indent=2, ensure_ascii=False)

print(f"Índice salvo em: {indice_path}")

Índice salvo em: dados/indice_filmes.json


### Ver o índice:

In [14]:
print("Filmes coletados:")
print()

for i, resumo in enumerate(indice, 1):
    print(f"{i:2}. {resumo['title']:40} {resumo['release_date'][:4]}  {resumo['vote_average']}/10")

Filmes coletados:



## Passo 5: Tratamento de Erros

Nem sempre tudo funciona perfeitamente. Vamos aprender a lidar com erros.

### Tipos de erros comuns:

1. **Filme não existe** (404)
2. **API key inválida** (401)
3. **Limite de requisições** (429)
4. **Problemas de rede** (timeout)

Vamos testar cada um:

### Teste 1: Filme que não existe

In [15]:
# Tentar buscar ID muito alto (provavelmente não existe)
filme_invalido = buscar_filme_por_id(999999999)

if filme_invalido is None:
    print("Como esperado, este filme não existe.")

Erro ao buscar filme 999999999: 401
Como esperado, este filme não existe.


### Função melhorada com tratamento de erros:

In [16]:
def buscar_filme_seguro(movie_id: int, max_tentativas: int = 3) -> dict:
    """
    Busca filme com tratamento de erros e tentativas.
    
    Args:
        movie_id: ID do filme
        max_tentativas: Número máximo de tentativas
    
    Returns:
        Dados do filme ou None se falhar
    """
    for tentativa in range(1, max_tentativas + 1):
        try:
            url = f"{config['tmdb']['base_url']}/movie/{movie_id}"
            
            response = requests.get(
                url,
                params={
                    'api_key': config['tmdb']['api_key'],
                    'language': config['tmdb']['language']
                },
                timeout=10  # Aguardar no máximo 10 segundos
            )
            
            if response.status_code == 200:
                return response.json()
            elif response.status_code == 404:
                print(f"Filme {movie_id} não encontrado")
                return None
            elif response.status_code == 429:
                print(f"Limite de requisições atingido, aguardando...")
                sleep(2)
            else:
                print(f"Erro {response.status_code} na tentativa {tentativa}")
                
        except requests.exceptions.Timeout:
            print(f"Timeout na tentativa {tentativa}")
        except requests.exceptions.RequestException as e:
            print(f"Erro de rede: {e}")
        
        # Aguardar antes de tentar novamente
        if tentativa < max_tentativas:
            sleep(1)
    
    return None

print("Função buscar_filme_seguro criada!")

Função buscar_filme_seguro criada!


### Testar função melhorada:

In [17]:
# Teste com filme válido
print("Teste 1: Filme válido")
filme = buscar_filme_seguro(550)
if filme:
    print(f"  Sucesso: {filme['title']}")
print()

# Teste com filme inválido
print("Teste 2: Filme inválido")
filme = buscar_filme_seguro(999999999)
if filme is None:
    print("  Tratado corretamente")

Teste 1: Filme válido
Erro 401 na tentativa 1
Erro 401 na tentativa 2
Erro 401 na tentativa 3

Teste 2: Filme inválido
Erro 401 na tentativa 1
Erro 401 na tentativa 2
Erro 401 na tentativa 3
  Tratado corretamente


## Passo 6: Estatísticas da Coleta

Vamos analisar os dados que coletamos.

### Contar arquivos na pasta dados:

In [18]:
arquivos_json = list(dados_path.glob("filme_*.json"))

print(f"Total de filmes salvos: {len(arquivos_json)}")

# Calcular tamanho total
tamanho_total = sum(f.stat().st_size for f in arquivos_json)

print(f"Tamanho total: {tamanho_total:,} bytes")
print(f"Tamanho médio: {tamanho_total // len(arquivos_json):,} bytes por filme")

Total de filmes salvos: 11
Tamanho total: 7,480 bytes
Tamanho médio: 680 bytes por filme


### Analisar notas dos filmes:

In [19]:
# Carregar índice
with open(dados_path / "indice_filmes.json") as f:
    indice = json.load(f)

# Calcular estatísticas
notas = [filme['vote_average'] for filme in indice]

print("Estatísticas das notas:")
print(f"  Maior nota: {max(notas)}/10")
print(f"  Menor nota: {min(notas)}/10")
print(f"  Média: {sum(notas) / len(notas):.2f}/10")

Estatísticas das notas:


ValueError: max() iterable argument is empty

### Filmes por década:

In [20]:
from collections import Counter

# Extrair décadas
decadas = []
for filme in indice:
    if filme['release_date']:
        ano = int(filme['release_date'][:4])
        decada = (ano // 10) * 10
        decadas.append(decada)

# Contar por década
contagem = Counter(decadas)

print("Filmes por década:")
for decada in sorted(contagem.keys()):
    print(f"  {decada}s: {contagem[decada]} filmes")

Filmes por década:


## Resumo do Bloco 2

### O que fizemos:

1. Descobrimos filmes através da lista de populares
2. Criamos funções para buscar filmes
3. Coletamos detalhes de múltiplos filmes
4. Implementamos tratamento de erros
5. Salvamos dados de forma organizada
6. Criamos um índice para consulta rápida
7. Analisamos estatísticas dos dados

### O que aprendemos:

- Diferença entre endpoints da API
- Como funciona paginação
- Por que usar funções
- Importância de rate limiting
- Como tratar erros em APIs
- Como organizar dados coletados

### Estrutura de arquivos:

```
dados/
  filme_550.json
  filme_603.json
  ...
  indice_filmes.json
```

## Verificação Final

In [21]:
print("Verificação Final")
print("=" * 50)

# Verificar pasta dados
if dados_path.exists():
    arquivos = list(dados_path.glob("filme_*.json"))
    print(f"Filmes salvos: {len(arquivos)}")
    
    # Verificar índice
    indice_existe = (dados_path / "indice_filmes.json").exists()
    print(f"Índice: {'OK' if indice_existe else 'NÃO encontrado'}")
    
    # Verificar funções
    print(f"Função buscar_filme_por_id: {'OK' if 'buscar_filme_por_id' in dir() else 'NÃO criada'}")
    print(f"Função buscar_filme_seguro: {'OK' if 'buscar_filme_seguro' in dir() else 'NÃO criada'}")
else:
    print("Pasta dados: NÃO encontrada")

print("=" * 50)
print()
print("Se tudo está OK, você está pronto para o próximo bloco!")

Verificação Final
Filmes salvos: 11
Índice: OK
Função buscar_filme_por_id: OK
Função buscar_filme_seguro: OK

Se tudo está OK, você está pronto para o próximo bloco!


## Próximo Passo

Agora que sabemos coletar vários filmes e tratar erros, vamos para o **Bloco 3**:

**03_estruturacao.ipynb** - Vamos organizar esses dados em estruturas Python (classes) para facilitar o trabalho.

### Pausa

Faça uma pausa de 5-10 minutos antes de continuar.
- Tome água
- Levante e estique
- Pense sobre o que coletamos